In [1]:
!pip install gdown
!pip install ultralytics

import csv
from ultralytics import YOLO
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
import gdown
import zipfile
import os
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import random
import shutil
import os
HOME = os.getcwd()
print(HOME)

!mkdir {HOME}/datasets
%cd {HOME}/datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.8/906.8 kB 23.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
/content
/content/datasets


In [2]:
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# ID file dari tautan
file_id = "1QDCT7w0TaG7hTu33Kcm-wBxGrsV9UEz9"
file_name = "ships-aerial-images"  # Nama file untuk menyimpan unduhan


try:
    print(f"Mengunduh file dengan ID {file_id}...")
    downloaded = drive.CreateFile({'id': file_id})
    downloaded.GetContentFile(file_name)
    print(f"File berhasil diunduh sebagai {file_name}!")
except Exception as e:
    print(f"Terjadi error saat mengunduh file: {e}")

Mengunduh file dengan ID 1QDCT7w0TaG7hTu33Kcm-wBxGrsV9UEz9...
File berhasil diunduh sebagai ships-aerial-images!


In [ ]:
!unzip /content/datasets/ships-aerial-images -d /content/data

In [10]:

import os

def count_items_in_directory(directory):
    try:
        return sum(len(files) for _, _, files in os.walk(directory))
    except FileNotFoundError:
        return 0

def count_items(base_path):
    categories = ['train', 'valid', 'test']
    classes = ["images", 'labels']
    total_images = 0

    for category in categories:
        print(f"--- {category.capitalize()} Data ---")
        category_path = os.path.join(base_path, category)
        for class_name in classes:
            class_path = os.path.join(category_path, class_name)
            num_items = count_items_in_directory(class_path)
            total_images += num_items
            if num_items > 0:
                print(f"{class_name.capitalize()}: {num_items} images")
            else:
                print(f"{class_name.capitalize()}: Directory does not exist or is empty")

    print(f"\nTotal Images: {total_images} images")

# Define base path
base_path = '/content/data/ships-aerial-images'
count_items(base_path)


--- Train Data ---
Images: 9697 images
Labels: 9697 images
--- Valid Data ---
Images: 2165 images
Labels: 2165 images
--- Test Data ---
Images: 1573 images
Labels: 1573 images

Total Images: 26870 images


In [17]:
from PIL import Image
import os

def check_images_in_directory(directory):
    invalid_images = []
    for root, _, files in os.walk(directory):
        for file in files:
            # Pastikan hanya file dengan ekstensi gambar yang diproses
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
                try:
                    img_path = os.path.join(root, file)
                    img = Image.open(img_path)
                    img.verify()  # Memverifikasi gambar (hanya memeriksa header)
                except (IOError, Image.UnidentifiedImageError) as e:
                    print(f"File tidak valid ditemukan: {img_path}, Error: {e}")
                    invalid_images.append(img_path)
                except Exception as e:
                    print(f"Unexpected error for {img_path}: {e}")
                    invalid_images.append(img_path)
    return invalid_images

# Periksa direktori gambar saja, abaikan folder "labels"
train_dir = "/content/data/ships-aerial-images/train/images"
validation_dir = "/content/data/ships-aerial-images/valid/images"

invalid_train_images = check_images_in_directory(train_dir)
invalid_validation_images = check_images_in_directory(validation_dir)

print("Invalid train images:", invalid_train_images)
print("Invalid validation images:", invalid_validation_images)


Invalid train images: []
Invalid validation images: []


In [ ]:
import os
from PIL import Image

def check_images_in_directory(directory):
    invalid_images = []
    for root, _, files in os.walk(directory):
        for file in files:
            try:
                img_path = os.path.join(root, file)
                img = Image.open(img_path)
                img.verify()
            except (IOError, Image.UnidentifiedImageError) as e:
                invalid_images.append((img_path, str(e)))
    return invalid_images

# memeriksa semua dataset
invalid_train_images = check_images_in_directory("/content/data/ships-aerial-images/train")
invalid_validation_images = check_images_in_directory("/content/data/ships-aerial-images/validation")

if invalid_train_images:
    print("Invalid images found in train dataset:")
    for img_path, error in invalid_train_images:
        print(f"File tidak valid ditemukan: {img_path}, Error: {error}")

if invalid_validation_images:
    print("Invalid images found in validation dataset:")
    for img_path, error in invalid_validation_images:
        print(f"File tidak valid ditemukan: {img_path}, Error: {error}")

In [19]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()


Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.3/112.6 GB disk)


In [20]:
from ultralytics import YOLO

from IPython.display import display, Image

os.environ['WANDB_MODE'] = 'disabled'

In [22]:
!yolo task=detect mode=train model=yolov9m.pt data="/content/data/ships-aerial-images/data.yaml" epochs=20 imgsz=256 batch=32 device=0 workers=8

100% 39.1M/39.1M [00:00<00:00, 123MB/s]
Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov9m.pt, data=/content/data/ships-aerial-images/data.yaml, epochs=20, time=None, patience=100, batch=32, imgsz=256, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cr